# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jun 9 2022 2:42PM,239281,10,Enova-10108,Emerginnova,Released
1,Jun 9 2022 2:42PM,239281,10,Enova-10109,Emerginnova,Released
2,Jun 9 2022 2:42PM,239281,10,Enova-10110,Emerginnova,Released
3,Jun 9 2022 2:35PM,239280,19,DEX0006980,Dexcel Pharma Technologies Ltd. DBA WDSrx,Released
4,Jun 9 2022 2:35PM,239280,19,DEX0006981,Dexcel Pharma Technologies Ltd. DBA WDSrx,Released
5,Jun 9 2022 2:35PM,239280,19,DEX0006982,Dexcel Pharma Technologies Ltd. DBA WDSrx,Released
6,Jun 9 2022 2:35PM,239280,19,DEX0006983,Dexcel Pharma Technologies Ltd. DBA WDSrx,Released
7,Jun 9 2022 2:28PM,239279,10,0085752472,ISDIN Corporation,Released
8,Jun 9 2022 2:28PM,239279,10,0085752348,ISDIN Corporation,Released
9,Jun 9 2022 2:28PM,239279,10,0085752374,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
34,239276,Released,4
35,239278,Released,28
36,239279,Released,23
37,239280,Released,4
38,239281,Released,3


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
239276,NaN,NaN,4.0
239278,NaN,NaN,28.0
239279,NaN,NaN,23.0
239280,NaN,NaN,4.0
239281,NaN,NaN,3.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
239172,0.0,1.0,0.0
239182,0.0,0.0,1.0
239188,0.0,0.0,1.0
239190,0.0,0.0,1.0
239201,17.0,16.0,6.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
239172,0,1,0
239182,0,0,1
239188,0,0,1
239190,0,0,1
239201,17,16,6


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,239172,0,1,0
1,239182,0,0,1
2,239188,0,0,1
3,239190,0,0,1
4,239201,17,16,6


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,239172,,1,
1,239182,,,1
2,239188,,,1
3,239190,,,1
4,239201,17,16,6


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jun 9 2022 2:42PM,239281,10,Emerginnova
3,Jun 9 2022 2:35PM,239280,19,Dexcel Pharma Technologies Ltd. DBA WDSrx
7,Jun 9 2022 2:28PM,239279,10,ISDIN Corporation
30,Jun 9 2022 2:26PM,239278,10,ISDIN Corporation
58,Jun 9 2022 2:09PM,239276,12,Hush Hush
62,Jun 9 2022 2:00PM,239248,20,"ACI Healthcare USA, Inc."
70,Jun 9 2022 1:50PM,239275,10,"Nextsource Biotechnology, LLC"
72,Jun 9 2022 1:46PM,239274,16,Sartorius Stedim North America
73,Jun 9 2022 1:39PM,239273,20,"Exact-Rx, Inc."
74,Jun 9 2022 1:39PM,239272,18,Innova Softgel LLC


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jun 9 2022 2:42PM,239281,10,Emerginnova,,,3
1,Jun 9 2022 2:35PM,239280,19,Dexcel Pharma Technologies Ltd. DBA WDSrx,,,4
2,Jun 9 2022 2:28PM,239279,10,ISDIN Corporation,,,23
3,Jun 9 2022 2:26PM,239278,10,ISDIN Corporation,,,28
4,Jun 9 2022 2:09PM,239276,12,Hush Hush,,,4
5,Jun 9 2022 2:00PM,239248,20,"ACI Healthcare USA, Inc.",,6,2
6,Jun 9 2022 1:50PM,239275,10,"Nextsource Biotechnology, LLC",,,2
7,Jun 9 2022 1:46PM,239274,16,Sartorius Stedim North America,,1,
8,Jun 9 2022 1:39PM,239273,20,"Exact-Rx, Inc.",,,1
9,Jun 9 2022 1:39PM,239272,18,Innova Softgel LLC,,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 9 2022 2:42PM,239281,10,Emerginnova,3,,
1,Jun 9 2022 2:35PM,239280,19,Dexcel Pharma Technologies Ltd. DBA WDSrx,4,,
2,Jun 9 2022 2:28PM,239279,10,ISDIN Corporation,23,,
3,Jun 9 2022 2:26PM,239278,10,ISDIN Corporation,28,,
4,Jun 9 2022 2:09PM,239276,12,Hush Hush,4,,
5,Jun 9 2022 2:00PM,239248,20,"ACI Healthcare USA, Inc.",2,6,
6,Jun 9 2022 1:50PM,239275,10,"Nextsource Biotechnology, LLC",2,,
7,Jun 9 2022 1:46PM,239274,16,Sartorius Stedim North America,,1,
8,Jun 9 2022 1:39PM,239273,20,"Exact-Rx, Inc.",1,,
9,Jun 9 2022 1:39PM,239272,18,Innova Softgel LLC,1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 9 2022 2:42PM,239281,10,Emerginnova,3,,
1,Jun 9 2022 2:35PM,239280,19,Dexcel Pharma Technologies Ltd. DBA WDSrx,4,,
2,Jun 9 2022 2:28PM,239279,10,ISDIN Corporation,23,,
3,Jun 9 2022 2:26PM,239278,10,ISDIN Corporation,28,,
4,Jun 9 2022 2:09PM,239276,12,Hush Hush,4,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 9 2022 2:42PM,239281,10,Emerginnova,3.0,NaN,NaN
1,Jun 9 2022 2:35PM,239280,19,Dexcel Pharma Technologies Ltd. DBA WDSrx,4.0,NaN,NaN
2,Jun 9 2022 2:28PM,239279,10,ISDIN Corporation,23.0,NaN,NaN
3,Jun 9 2022 2:26PM,239278,10,ISDIN Corporation,28.0,NaN,NaN
4,Jun 9 2022 2:09PM,239276,12,Hush Hush,4.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 9 2022 2:42PM,239281,10,Emerginnova,3.0,0.0,0.0
1,Jun 9 2022 2:35PM,239280,19,Dexcel Pharma Technologies Ltd. DBA WDSrx,4.0,0.0,0.0
2,Jun 9 2022 2:28PM,239279,10,ISDIN Corporation,23.0,0.0,0.0
3,Jun 9 2022 2:26PM,239278,10,ISDIN Corporation,28.0,0.0,0.0
4,Jun 9 2022 2:09PM,239276,12,Hush Hush,4.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2392434,73.0,11.0,11.0
12,478537,6.0,0.0,0.0
16,478532,1.0,1.0,0.0
18,1196121,5.0,0.0,0.0
19,717653,10.0,17.0,17.0
20,1435478,35.0,18.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2392434,73.0,11.0,11.0
1,12,478537,6.0,0.0,0.0
2,16,478532,1.0,1.0,0.0
3,18,1196121,5.0,0.0,0.0
4,19,717653,10.0,17.0,17.0
5,20,1435478,35.0,18.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,73.0,11.0,11.0
1,12,6.0,0.0,0.0
2,16,1.0,1.0,0.0
3,18,5.0,0.0,0.0
4,19,10.0,17.0,17.0
5,20,35.0,18.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,73.0
1,12,Released,6.0
2,16,Released,1.0
3,18,Released,5.0
4,19,Released,10.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,16,18,19,20
Status,,,,,,
Completed,11.0,0.0,0.0,0.0,17.0,0.0
Executing,11.0,0.0,1.0,0.0,17.0,18.0
Released,73.0,6.0,1.0,5.0,10.0,35.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,16,18,19,20
0,Completed,11.0,0.0,0.0,0.0,17.0,0.0
1,Executing,11.0,0.0,1.0,0.0,17.0,18.0
2,Released,73.0,6.0,1.0,5.0,10.0,35.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,16,18,19,20
0,Completed,11.0,0.0,0.0,0.0,17.0,0.0
1,Executing,11.0,0.0,1.0,0.0,17.0,18.0
2,Released,73.0,6.0,1.0,5.0,10.0,35.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()